In [38]:
import pandas as pd
import re

## Variable for storing minmum and maximum salary for different industry sector

In [39]:
salary_val = {'Banking_min': 45000, 'Banking_max':205000, 'Admin_min':45000, 'Admin_max':75000}

## Some initial cleaning work for filling nan and delete duplicate rows

In [40]:
#some initial data cleaning work
def initial_cleaning(df):
    df.fillna('unknown', inplace=True)
    df.drop_duplicates(inplace=True)

## A function used in location_area() to clean repeated string input

In [41]:
#a tool function for cleaning repeated
def apply_repeated(x):
    middle = len(x)//2
    if x != 'unknown':
        return x[:middle + 1]
    else:
        return 'unknown'

## Tidy the location and area column

In [42]:
#modify the location and area column
def location_area(df,location_index):
    df[['location','area']] = df[location_index].str.split('area:', expand = True)
    df.drop(location_index, axis = 1, inplace = True)
    df['area'].fillna('unknown', inplace = True)
    df['location'] = df['location'].apply(lambda x: x[9:])
    df['location'] = df['location'].apply(apply_repeated)
    df['area'] = df['area'].str.split(',', expand = True)[0]
    df['area'] = df['area'].apply(apply_repeated)

## Tidy the classification column

In [43]:
#modify the classification column
def classification(df, filename, classification_index, duplicate_index):
    if filename == 'NZ_Banking_JOBS.xlsx':
        df.drop(duplicate_index, axis = 1, inplace = True)
    df[['classification','Classification']] = df[classification_index].str.split('Classification:', expand = True)
    df['Classification'].fillna('unknown', inplace = True)
    df['Classification'] = df['Classification'].apply(apply_repeated)
    df['classification'] = df['classification'].apply(lambda x: x[15:] if 'classification' in x else 'unknown')
    df[['classification1','classification2','classification3']] = df['classification'].str.split('&', expand = True).fillna('unknown')

## Tidy the salary column

In [44]:
#deal with the salary column
def salary(df, filename, classification_index, min_val, max_val):
    df['salary'] = df[classification_index].apply(lambda x: 'unknown' if 'classification' in x else x)
    df.drop('classification', axis = 1, inplace = True)
    for i in range(10):
        for j in ['k','K']:
            df['salary'] = df[df['salary'].str.contains('\d',na = False)]['salary'].str.replace(str(i) + j,str(i) + '000', regex = True)
    df['salary'] = df[df['salary'].str.contains('\d', na = False)]['salary'].str.replace(' to ', '-', regex = True)
    df['salary'] = df[df['salary'].str.contains('\d', na = False)]['salary'].str.replace(',','')
    df['salary'].fillna('unknown', inplace = True)
    df[['min_salary','max_salary']] = df['salary'].str.split('-', expand = True)
    #deal with min_salary and max_salary
    df['min_salary'] = df['min_salary'].str.replace(' ','')
    df['max_salary'] = df['max_salary'].str.replace(' ', '')
    df['min_salary'] = df[df['min_salary'].str.contains('\d', na = False)]['min_salary'].str.extract('(\d+.?\d+)')
    df['max_salary'] = df[df['max_salary'].str.contains('\d', na = False)]['max_salary'].str.extract('(\d+.?\d+)')
    df['min_salary'].fillna(0, inplace = True)
    df['max_salary'].fillna(0, inplace = True)
    
    df['min_salary'] = df['min_salary'].astype('float')
    df['max_salary'] = df['max_salary'].astype('float')
    
    job_salary_fill_in(df, 'min_salary', min_val, max_val)
    job_salary_fill_in(df, 'max_salary', min_val, max_val)        
    
#     for r, row in enumerate(df['min_salary'].values):
#         #if there is no minmum salary value
#         if row == 0:
#             max_salary = df.loc[r, 'max_salary']
#             min_salary = np.random.randint(45000, 75000, size = 1)[0]
#             while min_salary > max_salary:
#                 min_salary = np.random.randint(45000, 75000, size = 1)[0]
#             df.loc[r,'min_salary'] = np.random.randint(45000, 75000, size = 1)[0]
            
#     for r, row in enumerate(df['max_salary'].values):
#         if row == 0:
#             df.loc[r,'max_salary'] = np.random.randint(45000, 75000, size = 1)[0]

    
    #transfer hourly salary to annual salary
    df.loc[df['min_salary'] < 1000, 'min_salary'] = df['min_salary']*8*200
    df.loc[df['max_salary'] < 1000, 'max_salary'] = df['max_salary']*8*200
    

## Fill in job_salary for all empty cells in salary columns

In [49]:
#fill in job_salary for all empty cells in salary columns
def job_salary_fill_in(df, minormax, min_val, max_val):
    print(minormax)
    import numpy as np
    ismin = False
    if minormax == 'min_salary':
        opposite = 'max_salary'
        ismin = True
    else:
        opposite = 'min_salary'
    for r, row in enumerate(df[minormax].values):
    #if there is no minmum salary value or maximum salary value
        if row == 0:
            if df.iloc[r][opposite] != 0:
                opposite_salary = df.loc[r, opposite]
            else:
                if ismin:
                    opposite_salary = max_val
                else:
                    opposite_salary = min_val

            minormax_salary = np.random.randint(min_val, max_val, size = 1)[0]
            
            if ismin:
                while minormax_salary > opposite_salary:
                    minormax_salary = np.random.randint(min_val, max_val, size = 1)[0]
            else:
                if opposite_salary == max_val:
                    minormax_salary = max_val
                else:
                    while minormax_salary < opposite_salary:
                        minormax_salary = np.random.randint(min_val, max_val, size = 1)[0]

            df.loc[r,minormax] = minormax_salary

## Main function

In [50]:
def clean_nz_seek(filename, location_index, classification_index, posted_time_index, duplicate_index):
    df = pd.read_excel(filename, header= None)
    initial_cleaning(df)
    location_area(df, location_index)
    classification(df, filename, classification_index, duplicate_index)
    
    if filename == 'NZ_Banking_JOBS.xlsx':
        min_val, max_val = salary_val['Banking_min'], salary_val['Banking_max']
    if filename == 'NZ_Admin_JOBS.xlsx':
        min_val, max_val = salary_val['Admin_min'], salary_val['Admin_max']
    
    salary(df, filename, classification_index, min_val, max_val)
    df.rename(columns={0:'Job Title', 1:'from website', 2:'company name', posted_time_index: 'posted_time', classification_index: 'classification'}, inplace = True)
    df.drop(index = [0], axis = 0, inplace = True)
    return df



In [47]:
%pdb on
df = clean_nz_seek('NZ_Banking_JOBS.xlsx', 
              location_index = 3,
              classification_index = 6, 
              posted_time_index = 5,
              duplicate_index = 4)

Automatic pdb calling has been turned ON
min_salary
max_salary


In [48]:
df

,Job Title,from website,company name,posted_time,classification,location,area,Classification,classification1,classification2,classification3,salary,min_salary,max_salary
1,Accounts Receivable,https://www.seek.co.nz/job/50568753?type=promo...,at MTF Finance Mt Wellington,"Featured,at",classification: Banking & Financial ServicesBa...,Auckland,Auckland Central,Credit,Banking,Financial ServicesBanking,Financial Servicessub,unknown,142406.0,172735.0
2,Internal Audit Manager,https://www.seek.co.nz/job/50556333?type=promo...,at Industrial and Commercial Bank of China (Ne...,"Featured,at",classification: Banking & Financial ServicesBa...,Auckland,Auckland Central,Compliance & Risk,Banking,Financial ServicesBanking,Financial Servicessub,unknown,134681.0,170782.0
3,Client Services Officer,https://www.seek.co.nz/job/50638706?type=stand...,at NZ Funds Management Limited,"8h ago,at",classification: Banking & Financial ServicesBa...,Auckland,Auckland Central,Client Services,Banking,Financial ServicesBanking,Financial Servicessub,unknown,45764.0,71391.0
4,Customer Banking Consultant - Invercargill,https://www.seek.co.nz/job/50637958?type=stand...,at Westpac,"1d ago,at",Salary + benefits + career progression,Southland,Invercargill,unknown,unknown,unknown,unknown,unknown,81030.0,186271.0
5,Private Wealth Assistant - Queenstown,https://www.seek.co.nz/job/50617226?type=stand...,at Craigs Investment Partners,"5d ago,at",classification: Banking & Financial ServicesBa...,Otago,Queenstown & Wanaka,Client Services,Banking,Financial ServicesBanking,Financial Servicessub,unknown,48008.0,166261.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3881,Senior Private Wealth Specialist,https://www.seek.co.nz/job/50526368?type=stand...,at Debbie Graham & Associates Limited,"20d ago,at",classification: Banking & Financial ServicesBa...,Auckland,unknown,Financial Planning,Banking,Financial ServicesBanking,Financial Servicessub,unknown,142487.0,146164.0
3882,Financial Adviser (Auckland),https://www.seek.co.nz/job/50501103?type=stand...,at Tyler Wren,"25d ago,at",Market Related base salary + Company benefits,Auckland,Auckland Central,unknown,unknown,unknown,unknown,unknown,52566.0,167600.0
3883,New Ventures Manager,https://www.seek.co.nz/job/50487878?type=stand...,at Matthew Wood Search,"27d ago,at",classification: Banking & Financial ServicesBa...,Auckland,Auckland Central,Corporate Finance & Investment Banking,Banking,Financial ServicesBanking,Financial Servicessub,unknown,71659.0,68039.0
3884,Regulatory and Compliance Manager-12 month con...,https://www.seek.co.nz/job/50575511?type=stand...,at Debbie Graham & Associates Limited,"12d ago,at",classification: Banking & Financial ServicesBa...,Auckland,Auckland Central,Compliance & Risk,Banking,Financial ServicesBanking,Financial Servicessub,unknown,0.0,0.0


In [173]:
df.iloc[3883][]

Job Title          Regulatory and Compliance Manager-12 month con...
from website       https://www.seek.co.nz/job/50575511?type=stand...
company name                   at Debbie Graham & Associates Limited
posted_time                                               12d ago,at
6                  classification: Banking & Financial ServicesBa...
location                                                    Auckland
area                                                Auckland Central
Classification                                     Compliance & Risk
classification1                                             Banking 
classification2                           Financial ServicesBanking 
classification3                                Financial Servicessub
salary                                                       unknown
min_salary                                                         0
max_salary                                                         0
Name: 3884, dtype: object